In [1]:
from utils import read_video, save_video, print_structure
from trackers import Tracker
import cv2
from team_assigner import TeamAssigner
from player_ball_assigner import PlayerBallAssigner

In [2]:
# Read Video
input_dir = 'input_video'
video_name = '08fd33_4.mp4'
# video_name = '0bfacc_0.mp4'
base_name = video_name.split('.')[0]
output_dir = 'output_video'

In [3]:
video_frames = read_video(f'{input_dir}/{video_name}')

# 初始化Tracker

In [4]:
tracker = Tracker('models/best_yolov5_100.pt')
# tracker = Tracker('models/train3_5xu/best.pt')
tracks = tracker.get_object_tracks(video_frames,
                                    read_from_stub=True,
                                    stub_path=f'stub/{base_name}.pkl')

# 对球插值

In [5]:
tracks['ball'] = tracker.interpolate_ball(tracks['ball'])

In [6]:
# 保存每个队员的截图, 只保存第一帧的，只一次
for id, player in tracks["players"][0].items():
    bbox = player["bbox"]
    frame = video_frames[0]
    cropped = frame[int(bbox[1]) : int(bbox[3]), int(bbox[0]) : int(bbox[2])]
    cv2.imwrite(f"{output_dir}/player_{id}.jpg", cropped)

# 分队

In [7]:
team_assigner = TeamAssigner()
team_assigner.assign_team(video_frames[0], 
                          tracks['players'][0])
for frame_num, player_track in enumerate(tracks['players']):
    for player_id, track in player_track.items():
        team = team_assigner.get_player_team(video_frames[frame_num], 
                                             track['bbox'],
                                                player_id)
        tracks['players'][frame_num][player_id]['team'] = team
        tracks['players'][frame_num][player_id]['team_color'] = team_assigner.team_colors[team]

tracker.team_colors = team_assigner.team_colors

# 分配控球

In [8]:
player_ball_assigner = PlayerBallAssigner()
team_control =[]
for frame_num, player_track in enumerate(tracks['players']):
    ball = tracks['ball'][frame_num][1]['bbox']
    assigned_player = player_ball_assigner.assign_ball_2_player(player_track, ball)
    if assigned_player:
        tracks['players'][frame_num][assigned_player]['has_ball'] = True
        team_control.append(tracks['players'][frame_num][assigned_player]['team'])
    else:    
        team_control.append(team_control[-1])
        

In [9]:
# print_structure(tracks)

In [10]:
# 画出输出
## 画出对象轨迹
output_video_frames = tracker.draw_annotation(video_frames, tracks, team_control)

In [11]:
# Save Video
save_video(output_video_frames, f'{output_dir}/{video_name}.avi')

Saving Video: 100%|██████████| 750/750 [00:10<00:00, 72.60it/s]

Video saved at output_video/08fd33_4.mp4.avi
